In [180]:
%pip install opencage pandas plotly requests pandas #install libraries


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [181]:
#import libraries here:
import requests
import pandas as pd

In [182]:
url = 'https://api.open-meteo.com/v1/forecast' # api url which im using a get request

def get_forecast_hourly(lat, long, forecast_d=5): #creating an api call for user's input
    params = {
        'latitude': lat,  
        'longitude': long, #have to convert city to lat,long
        'forecast_days': forecast_d, #number of days in forecast
        'temperature_unit': 'fahrenheit', # Change unit maybe later according to user pref
        'precipitation_unit': 'mm',
        'windspeed_unit': 'kmh', 
        'hourly': 'temperature_2m,precipitation',
        'timezone': 'America/New_York',  # Change according to your timezone - later user could maybe do this 
    }
    response = requests.get(url, params=params)
    return response.json()

In [183]:
from opencage.geocoder import OpenCageGeocode
key = '4e0e9443673d4093ab7e797721508ea3'  # my api key
geocoder = OpenCageGeocode(key) #gets lat,long from city

def get_cord(city):
    response =  geocoder.geocode(city)
    if  response:
        if len(response):
            return response[0]['geometry']['lat'], response[0]['geometry']['lng']
    else:
        print('city is not found')
        return None, None
    
# TODO: Add zip code search

city = input("Enter the city: ")  # user input
lat, long = get_cord(city)

if lat and long:
    forecast_data = get_forecast_hourly(lat, long)
    forecast_df = pd.DataFrame(forecast_data['hourly'])

#TODO: Add suggestions for city names
#TODO: Add error message if city input doesn't return anything

In [184]:
forecast_df #print out 5 day forecast per hour

,time,temperature_2m,precipitation_probability
0,2023-06-09 00:00:00,59.9,26
1,2023-06-09 01:00:00,59.8,13
2,2023-06-09 02:00:00,58.6,0
3,2023-06-09 03:00:00,58.0,2
4,2023-06-09 04:00:00,57.0,4
5,2023-06-09 05:00:00,56.5,6
6,2023-06-09 06:00:00,56.6,4
7,2023-06-09 07:00:00,58.3,2
8,2023-06-09 08:00:00,59.9,0
9,2023-06-09 09:00:00,61.7,0


Add in some functions which show user requests being used for visualization


In [185]:
def get_forecast_daily(lat, long, forecast_d): #creating an api call for user's input
    params = {
        'latitude': lat,  
        'longitude': long, #have to convert city to lat,long
        'forecast_days': forecast_d, #number of days in forecast
        'temperature_unit': 'fahrenheit', # Change unit maybe later according to user pref
        'precipitation_unit': 'mm',
        'windspeed_unit': 'kmh', 
        'daily': 'temperature_2m_max',
        'timezone': 'America/New_York',  # Change according to your timezone - later user could maybe do this 
    }
    response = requests.get(url, params=params)
    return response.json()

1) User jane asks for 10 days NYC

In [186]:
# visual example
city = input("Enter a city ")  
latitude, longitude = get_cord(city)
if latitude and longitude:
    data = get_forecast_daily(latitude, longitude,10)
    forecast_df = pd.DataFrame(data['daily'])

#TODO: Add suggestions for city names
#TODO: Add error message if city input doesn't return anything

In [187]:
#Create chart of temperature forecast for the next 10 days

import plotly.express as px
import plotly.graph_objects as go

forecast_df['time'] = pd.to_datetime(forecast_df['time']) #converting time column to a date/time data type
fig = px.line(forecast_df, x='time', y='temperature_2m_max') #defining x and y axis
fig.update_layout(
    title="Temperature Forecast",
    xaxis_title="Time",
    yaxis_title="Temperature (°F)",
)
fig.show()

#TODO: Add city name into title

2) Jane asks for rain and temp 

In [188]:
def get_forecast_daily(lat, long, forecast_d=10):
    params = {
        'latitude': lat,  
        'longitude': long,
        'forecast_days': forecast_d, 
        'temperature_unit': 'fahrenheit', 
        'precipitation_unit': 'mm',
        'windspeed_unit': 'kmh', 
        'daily': 'temperature_2m_min,temperature_2m_max,precipitation_probability_mean', #Added required parameters for precipitation
        'timezone': 'America/New_York',
    }
    response = requests.get(url, params=params)
    return response.json()

city = input("Enter a city ")  
latitude, longitude = get_cord(city)
if latitude and longitude:
    data = get_forecast_daily(latitude, longitude,10)
    forecast_df = pd.DataFrame(data['daily'])

#TODO: Add suggestions for city names
#TODO: Add error message if city input doesn't return anything

#dividing min and max to create an average temperature column
forecast_df['avg_temperature'] = (forecast_df['temperature_2m_min'] + forecast_df['temperature_2m_max']) / 2 
forecast_df['time'] = pd.to_datetime(forecast_df['time'])

fig = go.Figure()
fig.add_trace(go.Scatter(x=forecast_df['time'], y=forecast_df['temperature_2m_max'], mode='lines', line=dict(color='#e031e0'), name='Max Temp'))
fig.add_trace(go.Scatter(x=forecast_df['time'], y=forecast_df['temperature_2m_min'], mode='lines', line=dict(color='#30cf52'), name='Min Temp'))
fig.add_trace(go.Scatter(x=forecast_df['time'], y=forecast_df['avg_temperature'], mode='lines', line=dict(color='#7a31e0'), name='Avg Temp'))

high_precip = forecast_df[forecast_df['precipitation_probability_mean'] > 50] #defining what high precipitation is
low_precip = forecast_df[forecast_df['precipitation_probability_mean'] <= 50] #defining what low precipitation is

#adding precipitation to plot
fig.add_trace(go.Scatter(x=low_precip['time'], y=low_precip['precipitation_probability_mean'], mode='markers', marker=dict(color='#d1b900'), name='Low Precipitation'))
fig.add_trace(go.Scatter(x=high_precip['time'], y=high_precip['precipitation_probability_mean'], mode='markers', marker=dict(color='#31b5e8'), name='High Precipitation'))

fig.update_layout(
    title="Temperature Forecast",
    xaxis_title="Time",
    yaxis_title="Temperature (°F) / Precipitation Probability (%)",
    yaxis=dict(
        range=[0, 100],
    ),
)
fig.show()

#TODO: Add city name into title


3

In [189]:
def get_forecast_hourly_time(lat, long, forecast_d=1): #get the hourly forecast based on location input
    params = {
        'latitude': lat,  
        'longitude': long,
        'forecast_days': forecast_d, 
        'temperature_unit': 'fahrenheit', 
        'precipitation_unit': 'mm',
        'windspeed_unit': 'kmh', 
        'hourly': 'temperature_2m,precipitation_probability', 
        'timezone': 'America/New_York',
    }
    response = requests.get(url, params=params)
    forecast_data = response.json()
    forecast_df = pd.DataFrame(forecast_data['hourly']) #converting response into data frame
    forecast_df['time'] = pd.to_datetime(forecast_df['time'])
    return forecast_df

In [190]:
#creating city input
city = input("Enter a city name: ")  

latitude, longitude = get_cord(city)
if latitude and longitude:
    forecast_df = get_forecast_hourly_time(latitude, longitude)

#TODO: Add zip code input
#TODO: Add suggestions for city names
#TODO: Add error message if city input doesn't return anything


In [191]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=forecast_df['time'], y=forecast_df['temperature_2m'], mode='lines', line=dict(color='#7a31e0'), name='temperature')) #gives the temperature line graph for the hourly forecasr

high_precip = forecast_df[forecast_df['precipitation_probability'] > 50] 
low_precip = forecast_df[forecast_df['precipitation_probability'] <= 50]

fig.add_trace(go.Scatter(x=low_precip['time'], y=low_precip['precipitation_probability'], mode='markers', marker=dict(color='#d1b900'), name='low precipitation')) #scatter plot for precipitation based on hourly - assigning blue to low precipitation
fig.add_trace(go.Scatter(x=high_precip['time'], y=high_precip['precipitation_probability'], mode='markers', marker=dict(color='#31b5e8'), name='high precipitation')) #scatter plot for precipitation based on hourly - assigning red to high precipitation

fig.update_layout(
    title="Hourly forecast",
    xaxis_title="Time",
    yaxis_title="Temperature (°F) / Precipitation Probability (%)",
    
    yaxis=dict(
        range=[0, 100], #sets the range for temperature in fahrenheit & percentage of precipitation probability
    ),
)
fig.show() #shows hourly forecast

#TODO: Add city name into title
